In [51]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
import pickle

In [52]:
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d")
second_inn_feature_df = pd.read_csv('csv_data/feature_second_innings.csv',parse_dates=['match_date'],date_parser=custom_date_parser)

In [53]:
second_inn_feature_df.dropna(inplace=True)

In [54]:
result_df_train=pd.read_csv('csv_data/result_df_tran.csv',parse_dates=['match_date'],date_parser=custom_date_parser)
result_df_test=pd.read_csv('csv_data/result_df_test.csv',parse_dates=['match_date'],date_parser=custom_date_parser)
lr=pickle.load(open('first_innings_linear_regression.pkl','rb'))
scaler = pickle.load(open('first_innings_linear_regression_scaler.pkl','rb'))

In [55]:
feature_columns = ['team_score', 'opponent_score', 'location_base', 'location_mean','batsman_mean', 'batsman_max', 'bowler_mean']

In [56]:
second_innings_predict = lr.predict(scaler.transform(second_inn_feature_df[feature_columns]))

In [57]:
second_inn_result = pd.DataFrame(second_inn_feature_df)

In [58]:
second_inn_result['second_innings_prediction']=second_innings_predict

In [59]:
second_inn_result[['match_id','runs_scored','win','second_innings_prediction']].head()

,match_id,runs_scored,win,second_innings_prediction
0,366711,129,1,239.728980
1,378753,167,0,202.865693
2,378755,151,1,239.246302
3,351684,272,1,228.247344
4,351685,244,0,241.117383


In [60]:
final_result_test=result_df_test.merge(second_inn_result,on='match_id',how='inner')

In [61]:
final_result_test['predicted_win']=1*(final_result_test['second_innings_prediction']>final_result_test['first_innings_prediction'])

In [62]:
(final_result_test['predicted_win']==final_result_test['win']).sum()/final_result_test.shape[0]

0.7339449541284404

In [63]:
final_result_train=result_df_train.merge(second_inn_result,on='match_id',how='inner')
final_result_train['predicted_win']=1*(final_result_train['second_innings_prediction']>final_result_train['first_innings_prediction'])

In [64]:
(final_result_train['predicted_win']==final_result_train['win']).sum()/final_result_train.shape[0]

0.7475728155339806

In [65]:
final_result_train[['match_id','runs_scored_x','runs_scored_y','first_innings_prediction','second_innings_prediction','win','predicted_win']]

,match_id,runs_scored_x,runs_scored_y,first_innings_prediction,second_innings_prediction,win,predicted_win
0,351684,271,272,252.276262,228.247344,1,0
1,351685,249,244,266.305210,241.117383,0,0
2,385749,219,220,227.737850,312.299223,1,1
3,385750,290,161,236.115064,241.719478,0,1
4,378759,119,121,214.705063,253.447874,1,1
...,...,...,...,...,...,...,...
716,1157379,209,212,243.871615,270.152466,1,1
717,1144987,231,224,200.974774,271.975379,0,1
718,1144988,320,280,297.775833,208.376311,0,0
719,1153314,195,196,201.453530,276.961267,1,1


In [66]:
final_result_test.columns

Index(['match_id', 'match_date_x', 'team_x', 'opponent_x', 'location_x',
       'team_score_x', 'opponent_score_x', 'opponent_base_x',
       'opponent_trend_x', 'opponent_trend_predict_x', 'opponent_mean_x',
       'location_base_x', 'location_trend_x', 'location_trend_predict_x',
       'location_mean_x', 'current_base_x', 'current_trend_x',
       'current_trend_predict_x', 'current_mean_x', 'batsman_mean_x',
       'batsman_max_x', 'bowler_mean_x', 'bowler_max_x', 'is_train_x',
       'noise_x', 'runs_scored_x', 'first_innings_prediction', 'match_date_y',
       'team_y', 'opponent_y', 'location_y', 'team_score_y',
       'opponent_score_y', 'opponent_base_y', 'opponent_trend_y',
       'opponent_trend_predict_y', 'opponent_mean_y', 'location_base_y',
       'location_trend_y', 'location_trend_predict_y', 'location_mean_y',
       'current_base_y', 'current_trend_y', 'current_trend_predict_y',
       'current_mean_y', 'batsman_mean_y', 'batsman_max_y', 'bowler_mean_y',
       'bowl

In [67]:
final_result_test['win'].sum()

53

In [68]:
final_result_test.shape

(109, 56)

In [69]:
second_inn_feature_df[['match_id','team','opponent','win']]

,match_id,team,opponent,win
0,366711,New Zealand,West Indies,1
1,378753,Bangladesh,Zimbabwe,0
2,378755,Bangladesh,Sri Lanka,1
3,351684,South Africa,Australia,1
4,351685,South Africa,Australia,0
...,...,...,...,...
1074,1198240,Australia,England,1
1075,1233461,Zimbabwe,Pakistan,0
1076,1233462,Pakistan,Zimbabwe,1
1077,1223955,India,Australia,0
